# Reconhecimento de imagens: Twitter e Computer Vision API

## Bibliotecas básicas e outras inicializações

In [1]:
import tweepy as tw
%run credentials/credentials.py

## APP Twitter

### Autenticação

In [2]:
api_key = credentials['api_key']
api_secret_key = credentials['api_secret_key']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

### Primeiro tweet

In [3]:
auth = tw.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)
tweet = api.update_status("Python e Twitter na #Alura")

### Pesquisando tweets

In [6]:
tweets = tw.Cursor(api.search, q='data science', since='2021-01-01', lang='pt').items(5)
for tweet in tweets:
    print(tweet.user.screen_name)
    print(tweet.user.location)
    print(tweet.text)
    print('-----------------------------------------------------------------------------\n')

paanalytics1
São Paulo
2021-05-06 11:00:06.394061 - Consultoria em Data Science e Analytics: Análise de Dados, Machine Learning, Construçã… https://t.co/tfuMMCkuIK
-----------------------------------------------------------------------------

MesumRazaHemani
Karachi
TDataScience: A Data Science Template by @LucasEnkrateia 
https://t.co/JRScGxzDhg
-----------------------------------------------------------------------------

quarentenor
São Paulo
@Vazorlnd @StelaSalgado @BohrerIsadora @barbaramaria_ @oatila 5. você diz "a matéria da science não diz que a quebr… https://t.co/Yc3wlS4rX6
-----------------------------------------------------------------------------

Vazorlnd

@quarentenor @StelaSalgado @BohrerIsadora @barbaramaria_ @oatila calma lá companheiro... onde está sua educação?

a… https://t.co/SvppE0c8dN
-----------------------------------------------------------------------------

djhonyy
Florianópolis
@JorgeWill12 Recomendo a Alura e também a Data Science Academy.
--------------

### Pesquisando famosos

In [5]:
famosos = ['cauareymond', 'ivetesangalo', 'Tatawerneck']

for famoso in famosos:
    tweets = tw.Cursor(api.search, q=famoso, since='2019-01-01').items(20)
    print('Autor do tweet: ', famoso)
    print('Imagens postadas: ')
    for tweet in tweets:
        if 'media' in tweet.entities:
            print(tweet.entities['media'][0]['media_url'])
            
    print('---------------------------\n')

Autor do tweet:  cauareymond
Imagens postadas: 
http://pbs.twimg.com/media/E0oLkjIXsAUezqb.jpg
http://pbs.twimg.com/media/E0oLkjIXsAUezqb.jpg
http://pbs.twimg.com/media/E0oLkjIXsAUezqb.jpg
http://pbs.twimg.com/media/EzxFkuLXMAMgx-V.jpg
http://pbs.twimg.com/ext_tw_video_thumb/1389728594155425793/pu/img/_RIjRAFjkQDH6eEG.jpg
http://pbs.twimg.com/media/E0kumzaXIAELkh7.jpg
http://pbs.twimg.com/media/E0kumzaXIAELkh7.jpg
http://pbs.twimg.com/media/E0kumzaXIAELkh7.jpg
http://pbs.twimg.com/media/E0kumzaXIAELkh7.jpg
http://pbs.twimg.com/media/E0kumzaXIAELkh7.jpg
---------------------------

Autor do tweet:  ivetesangalo
Imagens postadas: 
http://pbs.twimg.com/media/E0ssCrUWEAgUDH-.jpg
http://pbs.twimg.com/media/E0rd9RhWUAEKHuC.jpg
---------------------------

Autor do tweet:  Tatawerneck
Imagens postadas: 
http://pbs.twimg.com/ext_tw_video_thumb/1390083584086749189/pu/img/LnkoLVEB_zQCrjyw.jpg
http://pbs.twimg.com/ext_tw_video_thumb/1390083584086749189/pu/img/LnkoLVEB_zQCrjyw.jpg
http://pbs.twimg

### Streaming

In [10]:
class MinhaStreamListener(tw.StreamListener):
    def on_status(self, status):
        print(status.user.screen_name)
        print(status.text)
        print('----------------------\n')
        

minhaStream = tw.Stream(auth=auth, listener=MinhaStreamListener())
minhaStream.filter(track='famosos')